# Enrichment analysys
### Over representation analysis
* Simplified ilustrative example
* Followed by real case
### GSEA
* Practical using python package

# Supervised machine learning
### Linear regression
### Logistic regression
### SVM
### Overfitting and Bias vs Variance -> Cross validation
### Dimentionality of the input -> PCA

# Unsupervised machine learning
### K-means clustering
* Using TCGA data, HR+/-

In [5]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

from ipywidgets import interact, interact_manual
from ipywidgets import IntSlider, FloatSlider, Dropdown, Text

interact_gen=interact_manual.options(manual_name="Generate data")
interact_plot=interact_manual.options(manual_name="Plot")
interact_calc=interact_manual.options(manual_name="Calculate tests")

In [ ]:
expression_data = pd.read_csv("data/TCGA_pan_data.tar.gz", index_col=0)
labels = pd.read_csv("data/TCGA_pan_labels.csv", index_col=0)

In [2]:
clinical_data = pd.read_csv('data/data_clinical_sample.txt', sep ='\t', index_col=0)
clinical_data = clinical_data.iloc[4:,:]
clinical_data = clinical_data.set_index("Sample Identifier")
expression_data = pd.read_csv('data/data_RNA_Seq_v2_expression_median.tar.gz', sep ='\t', index_col=0)
expression_data = expression_data.T.iloc[1:,:]

In [3]:
def multiple_gene_ttest(clinical_df, expression_df, separator, cond1, cond2):
    p_vec = []
    for gene in expression_df.columns:
        try:
            expression = expression_df[gene]
        except:
            print('Gene not found in data')
        try:
            group1 = clinical_df[separator] == cond1
            index1 = clinical_df[group1].index
            group2 = clinical_df[separator] == cond2
            index2 = clinical_df[group2].index
        except:
            print('Clinical condition wrong')
        expression1 = expression[index1].dropna()
        expression2 = expression[index2].dropna()
        p_val = sp.stats.ttest_ind(expression1, expression2).pvalue
        p_vec.append(p_val)
    return p_vec

def plot_hist_pi0_TCGA(stats, bins, threshold, pi0):
    plt.rcParams["figure.figsize"] = (20,10)
    stats = np.array(stats)
    stats_accepted = stats[stats <= threshold]
    m = len(stats)
    fdr_est = min((pi0*m*threshold)/len(stats_accepted),1)
    h = pi0*m/bins
    plt.hist(stats, bins = bins)
    plt.title('Estimated FDR: ' + str(np.round(fdr_est, 3)), fontsize=24)
    plt.vlines(threshold, 0, plt.gca().get_ylim()[1], color='r', linestyles = 'dashed')
    plt.hlines(h, 0, 1, color='black', linestyles = 'dashed', linewidth=3)
    plt.show()


def interact_multiple_gene_ttest(Criteria, Group_1, Group_2):
    global TCGA_multiple_t
    TCGA_multiple_t = multiple_gene_ttest(clinical_data, expression_data, Criteria, Group_1, Group_2)
    
def interactive_plot_hist_TCGA(Bins, Threshold, Pi0):
    plot_hist_pi0_TCGA(TCGA_multiple_t, Bins, Threshold, Pi0)

In [6]:
interact_calc(interact_multiple_gene_ttest, Criteria=Text('Surgical procedure first'), Group_1 = Text('Simple Mastectomy'), Group_2=Text('Lumpectomy'))
interact_plot(interactive_plot_hist_TCGA, Threshold = FloatSlider(min=0.01,max=0.5,value=0.05, step = 0.01, continuous_update=False), Bins=IntSlider(min=10,max=100, step = 10,value=20, continuous_update=False), Pi0 = FloatSlider(min=0,max=1, step = 0.01, continuous_update=False))

interactive(children=(Text(value='Surgical procedure first', description='Criteria'), Text(value='Simple Maste…

interactive(children=(IntSlider(value=20, continuous_update=False, description='Bins', min=10, step=10), Float…

<function __main__.interactive_plot_hist_TCGA(Bins, Threshold, Pi0)>

# References

Weinstein, John N., et al. 'The cancer genome atlas pan-cancer analysis project.' Nature genetics 45.10 (2013): 1113-1120.

Patrício, Miguel, et al. “Using Resistin, Glucose, Age and BMI to Predict the Presence of Breast Cancer.” BMC Cancer, vol. 18, no. 1, Jan. 2018, p. 29. BioMed Central, doi:10.1186/s12885-017-3877-1.